In [ ]:
from __future__ import annotations

import time
from datetime import datetime, timedelta, timezone

import numpy as np

from trading.contexts.backtest.adapters.outbound.config.backtest_runtime_config import (
    load_backtest_runtime_config,
)
from trading.contexts.backtest.application.dto import RunBacktestTemplate
from trading.contexts.backtest.application.services.close_fill_scorer_v1 import (
    CloseFillBacktestStagedScorerV1,
)
from trading.contexts.backtest.application.services.staged_runner_v1 import BacktestStagedRunnerV1
from trading.contexts.indicators.application.dto import CandleArrays
from trading.contexts.indicators.domain.entities import IndicatorId
from trading.contexts.indicators.domain.specifications import ExplicitValuesSpec, GridSpec
from trading.shared_kernel.primitives import (
    InstrumentId,
    MarketId,
    Symbol,
    Timeframe,
    TimeRange,
    UtcTimestamp,
)

try:
    import resource

    def rss_mb() -> float:
        # ru_maxrss is KB on macOS.
        return float(resource.getrusage(resource.RUSAGE_SELF).ru_maxrss) / 1024.0

except Exception:
    def rss_mb() -> float:
        return float('nan')


In [ ]:
def make_synth_candles(*, n: int = 10000, seed: int = 11) -> CandleArrays:
    rng = np.random.default_rng(seed)
    start = datetime(2024, 1, 1, tzinfo=timezone.utc)
    ts_open = (np.arange(n, dtype=np.int64) * 60_000) + int(start.timestamp() * 1000)

    noise = rng.normal(loc=0.0, scale=0.2, size=n).astype(np.float32)
    close = (200.0 + np.cumsum(noise)).astype(np.float32)
    open_ = np.roll(close, 1)
    open_[0] = close[0]
    spread = np.full(n, 0.08, dtype=np.float32)
    high = np.maximum(open_, close) + spread
    low = np.minimum(open_, close) - spread
    volume = (1200.0 + 100.0 * rng.random(n, dtype=np.float32)).astype(np.float32)

    end = start + timedelta(minutes=int(n))
    time_range = TimeRange(UtcTimestamp(start), UtcTimestamp(end))

    return CandleArrays(
        market_id=MarketId(1),
        symbol=Symbol("BTCUSDT"),
        time_range=time_range,
        timeframe=Timeframe("1m"),
        ts_open=ts_open.astype(np.int64),
        open=open_.astype(np.float32),
        high=high.astype(np.float32),
        low=low.astype(np.float32),
        close=close.astype(np.float32),
        volume=volume.astype(np.float32),
    )

candles = make_synth_candles()
candles.close.shape


In [ ]:
# This notebook is intended to run with the Numba compute engine.
from trading.contexts.indicators.adapters.outbound.compute_numba.engine import NumbaIndicatorCompute
from trading.contexts.indicators.domain.definitions import all_defs
from trading.platform.config.indicators_compute_numba import IndicatorsComputeNumbaConfig

numba_cfg = IndicatorsComputeNumbaConfig(
    numba_num_threads=1,
    numba_cache_dir=".cache/numba/notebooks",
    max_compute_bytes_total=1024 * 1024**2,
    max_variants_per_compute=600_000,
)
indicator_compute = NumbaIndicatorCompute(defs=all_defs(), config=numba_cfg)
numba_cfg


In [ ]:
# Moderate grid size smoke (tune windows list to scale variants).
windows = tuple(range(5, 205, 1))  # 200 variants
indicator_grid = GridSpec(
    indicator_id=IndicatorId("ma.sma"),
    source=ExplicitValuesSpec(name="source", values=("close",)),
    params={
        "window": ExplicitValuesSpec(name="window", values=windows),
    },
)

template = RunBacktestTemplate(
    instrument_id=InstrumentId(candles.market_id, candles.symbol),
    timeframe=candles.timeframe,
    indicator_grids=(indicator_grid,),
    indicator_selections=(),
    signal_grids=None,
    risk_grid=None,
    direction_mode="long-short",
    sizing_mode="all_in",
    risk_params=None,
    execution_params=None,
)

rt = load_backtest_runtime_config("configs/dev/backtest.yaml")
scorer = CloseFillBacktestStagedScorerV1(
    indicator_compute=indicator_compute,
    direction_mode=template.direction_mode,
    sizing_mode=template.sizing_mode,
    execution_params=template.execution_params or {},
    market_id=candles.market_id.value,
    target_slice=slice(200, candles.close.shape[0]),
    init_cash_quote_default=rt.execution.init_cash_quote_default,
    fixed_quote_default=rt.execution.fixed_quote_default,
    safe_profit_percent_default=rt.execution.safe_profit_percent_default,
    slippage_pct_default=rt.execution.slippage_pct_default,
    fee_pct_default_by_market_id=rt.execution.fee_pct_default_by_market_id,
    max_variants_guard=numba_cfg.max_variants_per_compute,
)

runner = BacktestStagedRunnerV1(parallel_workers=1)

rss0 = rss_mb()
t0 = time.perf_counter()
res = runner.run(
    template=template,
    candles=candles,
    preselect=50,
    top_k=20,
    indicator_compute=indicator_compute,
    scorer=scorer,
    defaults_provider=None,
    max_variants_per_compute=numba_cfg.max_variants_per_compute,
    max_compute_bytes_total=numba_cfg.max_compute_bytes_total,
    requested_time_range=candles.time_range,
    top_trades_n=3,
)
dt = time.perf_counter() - t0
rss1 = rss_mb()

variant_summary = (
    f"variants(stage_a_total={res.stage_a_variants_total}, "
    f"stage_b_total={res.stage_b_variants_total})"
)
print(variant_summary)
perf_summary = (
    f"top_k={len(res.variants)}  time={dt:.3f}s  "
    f"rss_mb_before={rss0:.1f}  rss_mb_after={rss1:.1f}"
)
print(perf_summary)

# Basic sanity.
assert len(res.variants) == 20
assert all(np.isfinite(v.total_return_pct) for v in res.variants)

[(v.variant_index, v.total_return_pct) for v in res.variants[:5]]
